In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import re

def save_and_cleanup(driver, df, modified_excel_path):
    # Save the modified DataFrame back to Excel
    df.to_excel(modified_excel_path, index=False)
    
    # Close the browser
    driver.quit()
    
    print("Task completed successfully. Modified Excel file saved.")

def scrape_and_extract(df, count, chrome_driver_path, modified_excel_path):
    # Create a Service object for the Chrome driver
    service = Service(chrome_driver_path)

    # Configure Chrome options
    options = Options()
    options.add_argument("--start-maximized")  # Start browser maximized

    # Initialize Chrome WebDriver with service and options
    driver = webdriver.Chrome(service=service, options=options)
    wait = WebDriverWait(driver, 20)  # Adjust wait time as needed

    try:
        loop_counter = 0

        for index, row in df.iterrows():
            if loop_counter >= count:
                break

            try:
                # Open the initial website
                driver.get('https://www.cms.gov/medicare-coverage-database/search.aspx')

                # Input search criteria (adjust according to your website)
                cpt_code = row['ID']
                word = 'denied'

                cpt_input = wait.until(EC.element_to_be_clickable((By.ID, 'tbxSearchBox')))
                cpt_input.clear()
                cpt_input.send_keys(cpt_code)

                # Perform the search
                search_button = wait.until(EC.element_to_be_clickable((By.ID, 'btnSubmitSearch')))
                search_button.click()
                
                # Handle initial pop-ups, if any
                try:
                    accept_button = wait.until(EC.element_to_be_clickable((By.ID, 'btnAcceptLicense')))
                    accept_button.click()
                except Exception as e:
                    print("No initial pop-up to accept or error accepting pop-up:", e)


                # Wait for results to load (adjust according to your website)
                wait.until(EC.presence_of_element_located((By.ID, 'h3ArticleGuidanceHeader')))

                # Extract paragraphs containing the target word
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                paragraphs = soup.find_all('p')  # Adjust tag based on actual structure

                similar_paragraphs = []
                for paragraph in paragraphs:
                    if re.search(r'\b{}\b'.format(re.escape(word)), paragraph.get_text(), re.IGNORECASE):
                        similar_paragraphs.append(paragraph.get_text())

                # Store results in DataFrame
                similar_paragraphs_str = '\n\n'.join(similar_paragraphs)
                df.at[index, 'Similar_Paragraphs'] = similar_paragraphs_str

                loop_counter += 1

            except Exception as e:
                print(f"Error processing row {index}: {e}")

    finally:
        save_and_cleanup(driver, df, modified_excel_path)

if __name__ == "__main__":
    excel_path = 'sample.xlsx'  # Replace with your input Excel file path
    chrome_driver_path = 'chromedriver.exe'  # Replace with your Chrome driver path
    modified_excel_path = 'modified_excel_file.xlsx'  # Replace with your desired output Excel file path
    
    # Load the Excel file
    df = pd.read_excel(excel_path)
    
    # Count the number of rows in the Excel file
    row_count = df.shape[0]
    
    # Scrape and extract data
    scrape_and_extract(df, row_count, chrome_driver_path, modified_excel_path)


In [ ]:
# extract that line alone

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import re

def save_and_cleanup(driver, df, modified_excel_path):
    # Save the modified DataFrame back to Excel
    df.to_excel(modified_excel_path, index=False)
    
    # Close the browser
    driver.quit()
    
    print("Task completed successfully. Modified Excel file saved.")

def scrape_and_extract(df, count, chrome_driver_path, modified_excel_path):
    # Create a Service object for the Chrome driver
    service = Service(chrome_driver_path)

    # Configure Chrome options
    options = Options()
    options.add_argument("--start-maximized")  # Start browser maximized

    # Initialize Chrome WebDriver with service and options
    driver = webdriver.Chrome(service=service, options=options)
    wait = WebDriverWait(driver, 20)  # Adjust wait time as needed

    try:
        loop_counter = 0

        for index, row in df.iterrows():
            if loop_counter >= count:
                break

            try:
                # Open the initial website
                driver.get('https://www.cms.gov/medicare-coverage-database/search.aspx')

                # Input search criteria (adjust according to your website)
                cpt_code = row['ID']
                word = 'denied'

                cpt_input = wait.until(EC.element_to_be_clickable((By.ID, 'tbxSearchBox')))
                cpt_input.clear()
                cpt_input.send_keys(cpt_code)

                # Perform the search
                search_button = wait.until(EC.element_to_be_clickable((By.ID, 'btnSubmitSearch')))
                search_button.click()
                
                # Handle initial pop-ups, if any
                try:
                    accept_button = wait.until(EC.element_to_be_clickable((By.ID, 'btnAcceptLicense')))
                    accept_button.click()
                except Exception as e:
                    print("No initial pop-up to accept or error accepting pop-up:", e)


                # Wait for results to load (adjust according to your website)
                wait.until(EC.presence_of_element_located((By.ID, 'h3ArticleGuidanceHeader')))

                # Extract lines containing the target word
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                lines = soup.find_all(text=True)

                similar_lines = []
                for line in lines:
                    if re.search(r'\b{}\b'.format(re.escape(word)), line, re.IGNORECASE):
                        similar_lines.append(line.strip())

                # Store results in DataFrame
                similar_lines_str = '\n\n'.join(similar_lines)
                df.at[index, 'Similar_Lines'] = similar_lines_str

                loop_counter += 1

            except Exception as e:
                print(f"Error processing row {index}: {e}")

    finally:
        save_and_cleanup(driver, df, modified_excel_path)

if __name__ == "__main__":
    excel_path = 'sample.xlsx'  # Replace with your input Excel file path
    chrome_driver_path = 'chromedriver.exe'  # Replace with your Chrome driver path
    modified_excel_path = 'modified_excel_file.xlsx'  # Replace with your desired output Excel file path
    
    # Load the Excel file
    df = pd.read_excel(excel_path)
    
    # Count the number of rows in the Excel file
    row_count = df.shape[0]
    
    # Add a column for storing similar lines
    df['Similar_Lines'] = ''
    
    # Scrape and extract data
    scrape_and_extract(df, row_count, chrome_driver_path, modified_excel_path)
